# Introduction
 For most of the cases, your data sources (or the website) wil provide an API for you to crawl data under their constraints. But somtimes, you are not provided with a good API, or you are not provided with any of API, for example a VR web chat room or a map related web application, you will need to find your own way to crawl the data. In this tuotrial I will discuss some advanced method to crawl data under the circumstances I mentioned above.
 
 ## Background
  First of all, we need to know that most of the mordern web applciations or websites are using RESTful to communicate with the background. And the one of the key points of the RESTful is stateless. That means if we can feed the proper payload into those APIs, we can get the data we need. Because the RESTful does not track what you have done before you sending the requests. Thus, if we can figure out the RESTful APIs and their payload requirements behind the screen, the rest are all about calling those APIs.
  
  ## Tools
  The tool we used is really handy, you don't actually need to install it. Most of the modern broswers, like chrome, firefox or Safari, come with a inspection tool. 
  
  Here's what it looks like in chrome when open google.com
  ![google.com](chrome.png)
  
  
  For those who usually do many web projects, they may be very familiar with this tool.For those who do not, I'm going to brefily introduce it. The inspection tool can be accessed by right clik any part of you web page and select inspect. 
  
  There are three features you need to care about:
  
  ### Elements: 
  You can view the entire html structure of a web page under this tag. It will highlight the rendered html element when you mouse is on the corresponding html section.
  
  ### Console:
  Everything that written by console.log() and exectuion errors in Javascript can be viewed under this tag
  
  ### Network: 
  The core feature which this tutorial relies on. It will show all the traffic between your broswer and the backend server, including file transmitting and API calls.
  
  
  
  

In [3]:
import requests
import json
import time
import datetime
import sys
import linecache

## Let's begin
  
  In this tutorial, we will use the livemap at [waze.com](https://www.waze.com/livemap) for example. 
  >Waze is the world's largest community-based traffic and navigation app. Join other drivers in your area who share real-time traffic and road info, saving everyone time and gas money on their daily commute.
  
  ###### (This webapp is now providing data api to developers, but we can still use it as an example in this tutorial)
  
  
  Navigate yourselves to a city that you are familiar with, for example New York. Then open your inspction tool and select Network section. Limit the results by clicking XHR, so we will see only the results of RESTful calls.
  
  ![XHR](XHR.png)
  
  Wait for a few seconds, and you will see something is popping up.
  
  Congratulation, you find the secret of the waze app. The treasure is hiding behind https://www.waze.com/rtserver/web/TGeoRSS (The api endpoint)
  
  For the next step, you need figure out what do those elements in the parameters and header mean. Unfortunately, I cannot provide a powerful tool to help you to figure out what are they, since there is not such tool.
  But thanks to the RESTful style, most of the parameters are readable. Let's go through these parameters.
  
  ![parameters](parameters.png)
 
  As you navigate through the map, you will find that top,bottom,left and right in the parameters are changing frequently. So these four parameters are the geo location that indicating where you are looking at. When you run follwoing codes, you will see that the respond comes with the following parts:
  ###### alerts
  ###### endTime
  ###### startTime
  ###### jams
  ###### users
  
  Thus, the meaning of parameters ma,mj and mu are very obvious, they are the maxuim numbers of alerts, jams, and users that will be returned.
  Finally, what does "_" mean in the parameter? As the responds keep poppin up, we can see that the "_" parameter is incrementing by one. So this parameter may be something that relates to the time.
  
  
  Now, it's your time to practice. Try modify or remove some of the parameters and see what is happening. You will find that the _ parameters is not necessary.

In [4]:
for i in range(10):
    try:
        r=requests.get("https://www.waze.com/rtserver/web/"+
                       "TGeoRSS?ma=600&mj=100&mu=100&left=-122.18939208984375"+
                       "&right=-121.56976318359375&bottom=37.3151955042825&"+
                       "top=38.32363990153987")
        my_js = json.loads(r.text)
        print(my_js.keys())
        break
    except Exception as e:
        time.sleep(1)
        
        #Why is the error here? Please continue reading
        # Please wait for seconds and rerun the code if you are not seeing the result
        print("error")
        continue

error
error
error
error
error
error
error
error
error
error


If you are lucky, you may see following result:  

dict_keys(['alerts', 'endTimeMillis', 'startTimeMillis', 'startTime', 'endTime', 'jams', 'users'])

## Sessions

Now you want to ask why the API failed in most of the times? Because we are using the unofficial method to get the data. Thus most of the requests are blocked. Inorder to work around this, we need to prevent like a broswer.Luckily the requests module in python provides a way that allow us the behave like a web broswer. This is called sessions.
>The Session object allows you to persist certain parameters across requests. It also persists cookies across all requests made from the Session instance, and will use urllib3's connection pooling  

The first step to be acting like a broswer, you need to specify the user-agent in the request header. This section will tell the server what broswer your are using. If you do not tell the server such information, the server will think you are not using any broswer, thus it will ban your request according to their policy.  

Then let's see how does the website work after we hit the enter button.

![login](login.png)  

As we can see from the screenshoot, the broswer first call a GET to www.waze.com/login/get before it starts loading the actual data. And in the respond header, we can see a Set-cookie section.  
The process here is clearer. The broswer will first call the /login/get to set the cookies before retriving data. The following data forge this call and store the cookies in the session object.

In [7]:
s = requests.session()
s.headers.update({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'+
                  'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
                  
r = s.get("https://www.waze.com/login/get",
          headers={'referer': 'https://www.waze.com/livemap'})

print("##############The header of first call\n\n")
for k,v in r.headers.items():
    print(k,v)
print("\n\n##############The header of the second call\n\n")
r=s.get("https://www.waze.com/rtserver/web/TGeoRSS?"+
        "ma=600&mj=100&mu=100&left=-122.18939208984375&"+
        "right=-121.56976318359375&bottom=37.3151955042825&"+
        "top=38.32363990153987")
for k,v in r.headers.items():
    print(k,v)
print(r)
s.close()

##############The header of first call


Server nginx
Date Fri, 30 Mar 2018 22:47:42 GMT
Content-Type text/json;charset=utf-8
Transfer-Encoding chunked
Set-Cookie _web_session=cllSMVhPSVErUGZGVFBWL3U1VEcwNGxiMDlzOTQ3aUszMVRvdEFiTG1Idz0tLVNLZGc3NGxlcTE4dDg4OTJaRlk1Vmc9PQ%3D%3D--bd7ad9586c801393434fa849401dd5274b4a0b3d; Expires=Sat, 31-Mar-2018 22:47:42 GMT; Path=/; Secure, _csrf_token=b1DlAze1FuoPmtoOrhsw78xOGe09NSYwkuuwvhEvcYA; Expires=Sat, 31-Mar-2018 22:47:42 GMT; Path=/; Secure
Strict-Transport-Security max-age=31536000; includeSubDomains;, max-age=31536000; includeSubDomains;
Alt-Svc clear, hq=":443"; ma=2592000; quic=51303432; quic=51303431; quic=51303339; quic=51303335,quic=":443"; ma=2592000; v="42,41,39,35"
X-Frame-Options SAMEORIGIN
X-Content-Type-Options nosniff
Via 1.1 google, 1.1 google


##############The header of the second call


Server nginx
Date Fri, 30 Mar 2018 22:47:42 GMT
X-Frame-Options SAMEORIGIN
Strict-Transport-Security max-age=31536000; includeSubDomains;
X-Co

But we are still getting error.Why?  

The part we did not cover is the headers part. And now it's time to talk about it. Header is another part that the server will check for the integrity of requests. Let's fill the data into our request header.  

![header](header.png)

Actually, we only need to add the "Host" and "refer" into the headers.Because the cookie part is automatically added when we use the session object to make the call. And the user-agent is already put into the session object. Those the headers content starting with ":" can be ignored (If you are interested, check here for why [link](https://stackoverflow.com/questions/30900008/purpose-of-pseudo-colon-header-fields). And all the rest will be added automatically if we are using the session object.

After filling in all of those part, we got a steady process to crawl the data.

In [14]:
s = requests.session()
s.headers.update({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'+
                  ' AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})

r = s.get("https://www.waze.com/login/get",headers={'referer': 'https://www.waze.com/livemap'})

headers={'Host':"www.waze.com",'referer': 'https://www.waze.com/livemap'}
r=s.get("https://www.waze.com//row-rtserver/web/TGeoRSS?"+
        "ma=600&mj=100&mu=100&left=-130.27148437499997&"+
        "right=-60.80273437500001&bottom=-35.6050948577363"+
        "&top=90.07903474330757",headers=headers)

print("\n\n############### result header")
for k,v in r.headers.items():
    print(k,v)
    
print("\n\n################ result \n\n")
my_js = json.loads(r.text)
print(my_js.keys())

print("\n\n################ cookies after the request")
print(s.cookies.items())
s.close()



############### result header
Server nginx
Date Sat, 31 Mar 2018 21:41:50 GMT
Content-Type text/x-json;charset=UTF-8
Cache-Control max-age=60
Content-Encoding gzip
Expires Sat, 31 Mar 2018 21:42:49 GMT
Last-Modified Sat, 31 Mar 2018 21:41:49 GMT
X-Proxy-Cache MISS
X-RT-NAME 231
X-Frame-Options SAMEORIGIN
Strict-Transport-Security max-age=31536000; includeSubDomains;
X-Content-Type-Options nosniff
Via 1.1 google
Alt-Svc hq=":443"; ma=2592000; quic=51303432; quic=51303431; quic=51303339; quic=51303335,quic=":443"; ma=2592000; v="42,41,39,35"
Transfer-Encoding chunked


################ result 


dict_keys(['alerts', 'endTimeMillis', 'startTimeMillis', 'startTime', 'endTime', 'jams', 'users'])


################ cookies after the request
[('_csrf_token', 'BBWJK9_RU6dgd6nhRmy0d1Jme_I0mD_6DAsFkkk8aX0'), ('_web_session', 'dmtHNU1McndZSnhUQ1Q3TGd6bFYwdFowdUJrendRQzVWbnBnL1Zhb3RYST0tLTVHUDZNMVhlSVV4T0gwMjdMNGdkbnc9PQ%3D%3D--259466b1b503d19ce4175e8e66b0e809ee0e2bde')]




## Redirection

Redirection is another useful feature that comes with the session object. I will allow the request to be redirect to other url. In some cases, for example, analyzing a set of shortened url, it will save ton's of time.

The follwoing code is try to get the real url hide behind the short url.

In [21]:
def get_real_url(s_url):
    s=requests.session()
    r=s.get(url=s_url,allow_redirects=True)
    s.close()
    return r.url
get_real_url("http://goo.gl/PffZZG")

'http://www.google.com/'